# Seattle Lidar

## Visualize Lidar Scattered Point Elevation Data

This notebook uses hvPlot and Datashader to visualize Lidar elevation data from [the Puget Sound Lidar consortium](https://pugetsoundlidar.ess.washington.edu/), a source of Lidar data for the Puget Sound region of the state of Washington, U.S.A.

### Lidar Elevation Data

Example X,Y,Z scattered point elevation data from the unpacked 7zip files (unpacked as .gnd files)

```
! head ../data/q47122d2101.gnd
```
```
X,Y,Z
1291149.60,181033.64,467.95
1291113.29,181032.53,460.24
1291065.38,181035.74,451.41
1291113.16,181037.32,455.51
1291116.68,181037.42,456.20
1291162.42,181038.90,467.81
1291111.90,181038.15,454.89
1291066.62,181036.73,451.41
1291019.10,181035.20,451.64
```

The Seattle area example below loads 25 `.gnd` elevation files like the one above. We'll download, cache and read the data using `intake`.

**NOTE**: Downloading the data for the first time takes about 2 mins due to its large size.

In [ ]:
import intake

cat = intake.open_catalog('./catalog.yml')
list(cat)

In [ ]:
lidar = cat.seattle_lidar()
ddf_original = lidar.to_dask()
print(len(ddf_original))
ddf_original.head(2)

### Geographic Metadata

Since the data are geographic, we need to know the coordinate reference system (CRS) of the X and Y. All the geographic metadata is stored in the data source entry in the intake catalog.

In [ ]:
lidar.metadata['crs']

We intend to display the data overlayed on top of a web basemap (aka tiled web map) which usually uses the Web Mercator (EPSG:3857) projection. We convert the data now to this projection so we only have to do it once. Note that we could have installed [GeoViews](https://geoviews.org/) and called hvPlot with `geo=True` to enable on-the-fly projection of the data to the right system. However, since we are going to generate many plots in this notebook, it is worth pre-processing the data beforehand not to pay the projection cost on every new plot.

In [ ]:
from pyproj.transformer import Transformer

# Washington State Plane North EPSG code and Mercator projection EPSG code
transformer = Transformer.from_crs('epsg:2855','epsg:3857')

In [ ]:
FT_2_M = 0.3048 # conversion factor from feet to meters

def convert_coords(df):
    lon, lat = transformer.transform(df.X.values * FT_2_M, df.Y.values * FT_2_M)
    df['meterswest'], df['metersnorth'] = lon, lat
    return df[['meterswest', 'metersnorth', 'Z']]

Try out the convert_coords function on a subset of the data

In [ ]:
convert_coords(ddf_original.head(2))

### Convert the coordinates

Since our real dataset is large and partitioned using dask, we need to think about how to apply the `convert_coords` function to our data. 

In [ ]:
import dask
import dask.distributed
import dask.delayed
from dask.config import set

set({'distributed.worker.memory.target': 0.8,
     'distributed.worker.memory.spill': 0.9,
     'distributed.worker.memory.pause': 0.95,
     'distributed.worker.memory.terminate': 0.98})

In [ ]:
dask.distributed.Client(memory_limit='12GB')

Explore the task graph to figure out a performant way to split up the coords conversion. First we'll try with using `dask.delayed`.

In [ ]:
dask.delayed(convert_coords)(ddf_original).visualize(engine='cytoscape')

We can see that even though we thought `dask.delayed` would help, in actuality we would be requiring all the processes to be done first and then the conversion would happen on the whole dataset in one go. Another approach would be to use `dask.map_partitions` to do the conversion on each piece of the data.

In [ ]:
ddf = ddf_original.map_partitions(convert_coords)
ddf.visualize(engine='cytoscape', tasks=True)

Now that we have set up the task graph, we can use `ddf` directly to do the computations on the fly. However, since this dataset fits in memory (~8 GB), we will do the computation and keep the output in memory for quick use when plotting.

**NOTE:** This next cell takes about a minute to run.

In [ ]:
%%time
df = ddf.compute()

In [ ]:
df.info()

If all the data doesn't fit in memory on your machine, try downsampling the data from each file to only keep 1/100th of the total data. To avoid unnecessary computation, it is better to do the downsampling first and _then_ convert the coordinates.

In [ ]:
ddf_small = ddf_original.sample(frac=0.01).map_partitions(convert_coords)
ddf_small.visualize(engine='cytoscape', tasks=True)

In [ ]:
%%time
df_small = ddf_small.compute()

### Plot

In [ ]:
import hvplot.dask # noqa
import hvplot.pandas # noqa

To simplify the exploration of the time required to display different data, define a function that accepts a regular `pandas` dataframe or a `dask` delayed dataframe. 

In [ ]:
def plot(data, **kwargs):
    """Plot point elevation data, rasterizing by mean elevation"""
    options = dict(cmap='blues_r', width=800, height=800, xaxis=None, yaxis=None, tiles='ESRI')
    options |= kwargs
    return data.hvplot.points('meterswest', 'metersnorth', color='Z', rasterize=True,
                              aggregator='mean', **options)

Then we will construct the plot using the `df_small` dataset. 

### Benchmarking

Here, we measure the execution time of the `display` function from IPython. Without including `display`, the timing would only reflect the rapid creation of the HoloViews `DynamicMap` objects. By including `display`, we also capture the duration of various internal Python operations required to render the plot on the screen, such as the rasterization process.

In [ ]:
from IPython.display import display

In [ ]:
%%time

display(plot(df_small))

In [ ]:
%%time

display(plot(df))

In [ ]:
%%time

display(plot(ddf))

## Visualize Raster of Point Data

If we compute a raster of the point data then we don't need to store as much data in memory, which should allow faster interactions, and allow use with lots of other tools.

In [ ]:
%%time
raster = plot(df, dynamic=False, width=1000, height=1000, tiles=None).data

The result is an `xarray.Dataset` with x and y coordinates and a 2D array of Z values.

In [ ]:
raster.metersnorth[1] - raster.metersnorth[0]

With these data we can use the geo tools found in the `xarray-spatial` library to compute and visualize the elevation using hillshading for instance. See [Datashader User Guide](https://datashader.org/user_guide/Geography.html) for more datashader and `xarray-spatial` geo tooling.

In [ ]:
from xrspatial import hillshade
import hvplot.xarray # noqa

In [ ]:
illuminated = hillshade(raster.get('meterswest_metersnorth Z'))
illuminated.hvplot.image(
    'meterswest', 'metersnorth', data_aspect=1, cmap='blues', tiles='ESRI',
    width=600, height=600, padding=0, xlabel='Longitude', ylabel='Latitude', colorbar=False,
)

## Ideas for future work

It'd be nice to have a `rasterio` writer from `xarray` so that we could easily write chunked geotiffs from `xarray` objects.

Something like:

```python
raster.to_rasterio(path=None, mode='w', compute=True)
```